<a href="https://colab.research.google.com/github/Adityeah18/tensorflow/blob/main/sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries

In [3]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses


#Downloading Dataset

In [4]:
print(tf.__version__)
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(
    "aclImdb_v1", url, untar=True, cache_dir=".", cache_subdir=""
)  # Downloading and extracting
# dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb') #This path given by Documentation is not working
dataset_dir = os.path.join(dataset, "aclImdb")
train_dir = os.path.join(
    dataset_dir, "train"
)  # directory for train set in the downloaded data

2.18.0
84125825/84125825 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


Viewing the dataset

In [5]:
#To see the Review at position 1181, of Positive train set
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())


Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [6]:
# random review from the positive set
import random

pos_dir = os.path.join(train_dir, "pos")
sample_filename = random.choice(os.listdir(pos_dir))
sample_path = os.path.join(pos_dir, sample_filename)

with open(sample_path) as f:
    print(f"\nReview: {sample_filename}\n")
    print(f.read())


Review: 11082_10.txt

I watched this movie when I was almost quite a kid, and, naturally, was moved to tears by this story of a fox family. The fantastic scenery at Hokkaidô, the excellent storytelling and last not least the wonderful soundtrack provide a rare intimacy with the protagonists. I am still searching for some copy of the gorgeous soundtrack. To German viewers it might be useful to know that the DEFA-dubbing is the only one worth listening to. I taped both (DEFA and BR) but I keep viewing the first one only.


In [7]:
# Remove the directory for the unsupervised dataset from 'train_dir'
remove_dir = os.path.join(
    train_dir, "unsup"
)  # Removing the file directory of the unsupervised set
shutil.rmtree(remove_dir)  # This deletes the 'unsup' directory and all its contents

# Batch size for training. Determines the number of samples processed in one step.
batch_size = 32  # A batch size of 32 is common, but it may vary depending on your system and dataset.

# Seed for reproducibility. This ensures the random operations (like data shuffling) are consistent across runs.
seed = 42  # A specific seed value is chosen to make the random processes deterministic.

# Loading the raw training dataset with text files in the directory
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir,  # The directory where the training data is located (after removing 'unsup')
    batch_size=batch_size,  # Number of samples to return in each batch
    validation_split=0.2,  # 20% of the data will be used for validation
    subset="training",  # Indicating that this is the training subset (remaining 80% is training data)
    seed=seed,
)  # Using the seed for random operations (like splitting the data)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


Example of 1st 3 reviews in training set

In [8]:
# Iterate through the raw_train_ds dataset, and take the first batch
for text_batch, label_batch in raw_train_ds.take(1):
    # 'take(1)' retrieves the first batch of data (text and labels)

    # Loop through the first 3 items in the batch
    for i in range(3):
        # Print the text review (converted from tensor to numpy array for readability)
        print("Review", text_batch.numpy()[i])

        # Print the corresponding label (converted from tensor to numpy array for readability)
        print("Label", label_batch.numpy()[i])

Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

in the review text we seen tags like line breakand all other stuff,
Also the labels which are targets tells 0 and 1 , to see what those 0 and 1 means see below:

In [9]:
print(
    "Label 0 correponds to", raw_train_ds.class_names[0]
)  # Displaying the name of class at index 0
print(
    "Label 1 corresponds to", raw_train_ds.class_names[1]
)  # Displaying the name of class at index

Label 0 correponds to neg
Label 1 corresponds to pos


we have seen the 0 = negative review1=Positive review


#Test-Train Split

now we train the 80 %  and test the remaining 20 % which makes:
80% train
20% test

In [10]:
# Load the validation dataset
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir,  # Use train_dir variable which has correct path
    batch_size=batch_size,
    validation_split=0.2,  # 20% of the training data will be used for validation
    subset="validation",  # Specifies this is for validation
    seed=seed,  # Ensures reproducibility
)

# Define the directory for the test data
test_dir = os.path.join(dataset_dir, "test")

# Load the test dataset
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    test_dir,  # Now use the defined test_dir variable
    batch_size=batch_size,  # The batch size for the test dataset
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


see th 5000 is used for test of dataset

#Preperation of Dataset

For data preperation the we have to make the dataset CLAEAN:
for that we have to

1.   Standardize=remove punct and </ br> of HTML
2.   Tokennize = splitint the words with whitespaces
3.   Vectorize = converting str into number
For all this we have keras.TextVectorization in layers

In [11]:
from tensorflow.keras.layers import TextVectorization  # Correct import


def custom_standardization(input_data):
    # Convert input to lowercase
    lowercase = tf.strings.lower(input_data)
    # Remove any HTML tags like <br />
    stripped_html = tf.strings.regex_replace(
        lowercase, "<br />", " "
    )  # Fixed: Replaced '<br />' with '<br />' and added space
    # Remove punctuation using regex
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )  # Fixed: Changed tf.string to tf.strings


# Parameters for vectorization
max_features = 10000  # The number of unique words to keep (vocabulary size)
sequence_length = 250  # The length of the input sequences (padding/truncating)

# Create the TextVectorization layer
vectorize_layer = TextVectorization(  # Changed layers.Vectorization to TextVectorization
    standardize=custom_standardization,  # Custom standardization function
    max_tokens=max_features,  # Maximum number of words to consider
    output_mode="int",  # Output integers (indices of words in vocabulary)
    output_sequence_length=sequence_length,  # The length of each sequence (padding or truncating will occur)
)

making the text only dataset

In [12]:
# Extract only text (input data) from the training dataset
train_text = raw_train_ds.map(lambda x, y: x)

# Adapt the vectorization layer to the training data (calculating the vocabulary)
vectorize_layer.adapt(train_text)

Creating a function for this vectorize text

In [13]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)  # don know what expand is doing??
    return vectorize_layer(text), label


# getting the batch of 59 reviews and labes from dataset
text_batch, label_batch = next(iter(raw_train_ds))  # ???
# reading the only 1st review and then reading its label and showing
first_review, first_label = text_batch[0], label_batch[0]
print("Review:", first_review)
print("Label:", raw_train_ds.class_names[first_label])
print("Vectorized review:", vectorize_text(first_review, first_label))
# Why above is retriving the 32 reviews and 32 labels? inever mention 32?

# Printing the selected integer vectorization text into its text
print("1287", vectorize_layer.get_vocabulary()[1287])
print("313", vectorize_layer.get_vocabulary()[313])

Review: tf.Tensor(b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />

#Training dataset

In [14]:
# Mapping the vectorize_text function to each dataset
x_train = raw_train_ds.map(vectorize_text)
x_val = raw_val_ds.map(vectorize_text)
x_test = raw_test_ds.map(vectorize_text)

# Cache the datasets to speed up access
# This stores the dataset in memory after the first pass, preventing it from being read multiple times from disk
# The prefetch method prepares the next batch of data while the model is training
AUTOTUNE = tf.data.AUTOTUNE

x_train = x_train.cache().prefetch(buffer_size=AUTOTUNE)
x_val = x_val.cache().prefetch(buffer_size=AUTOTUNE)
x_test = x_test.cache().prefetch(buffer_size=AUTOTUNE)

Creating a model= Neural Network

In [58]:
embedding_dim = 16  # Dimensionality of the word embeddings

model = tf.keras.Sequential(
    [
        layers.Embedding(
            max_features, embedding_dim
        ),  # Embedding layer transforms words into vectors of size `embedding_dim`
        layers.Dropout(
            0.2
        ),  # Dropout to reduce overfitting by ignoring 20% of input randomly
        layers.GlobalAveragePooling1D(),  # Pooling to average the word embeddings across the sequence
        layers.Dropout(0.2),  # Another Dropout layer for further regularization
        layers.Dense(
            1, activation="sigmoid"
        ),  # Sigmoid activation for binary classification (output between 0 and 1)
    ]
)

model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_2      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

#Compilation of model

In [59]:
model.compile(
    optimizer="adam",  # popular optimizer to optimize weights and biases
    loss=losses.BinaryCrossentropy(),  # Loss function for binary classification
    metrics=[
        tf.metrics.BinaryAccuracy(threshold=0.5) #Threshold is the line b/w the good and bad; here if model have accuracy of vector >50 then its a good word in context and vice vers
    ],  # Metrics to evaluate model performance
)



#Fitting Model


In [60]:
history=model.fit(x_train,validation_data=x_val,epochs=10)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - binary_accuracy: 0.5803 - loss: 0.6817 - val_binary_accuracy: 0.7312 - val_loss: 0.6120
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - binary_accuracy: 0.7643 - loss: 0.5783 - val_binary_accuracy: 0.8090 - val_loss: 0.4971
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - binary_accuracy: 0.8262 - loss: 0.4642 - val_binary_accuracy: 0.8344 - val_loss: 0.4247
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - binary_accuracy: 0.8515 - loss: 0.3946 - val_binary_accuracy: 0.8374 - val_loss: 0.3878
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - binary_accuracy: 0.8674 - loss: 0.3474 - val_binary_accuracy: 0.8382 - val_loss: 0.3678
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - binary_accuracy: 0.8811 - loss: 0.3139 - val_binary_accuracy: 0.8552 - val_loss: 0.3425
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - binary_accuracy: 0.8928 - loss: 0.2896 - val_binary_accuracy: 0.8546 - val_loss: 0.3325
Epoch 

#Evaluate

In [61]:
loss, accuracy = model.evaluate(x_test, verbose=2)
# Evaluates the model on the test dataset (x_test)
# 'verbose=2' prints one line per epoch: more informative than 0 (silent) or 1 (progress bar)

print("Loss", loss)  # Displays how far off the model's predictions are (lower is better)
print(f"Accuracy {accuracy:.3f}")  # Displays accuracy as a percentage (formatted to 3 decimal places)


782/782 - 2s - 3ms/step - binary_accuracy: 0.8555 - loss: 0.3319
Loss 0.33192166686058044
Accuracy 0.855


#Predicting

To create new model with the trained sets which we did till now and with that, vectorize_layer and if want then change the activation

In [63]:
export_model = tf.keras.Sequential([
    vectorize_layer,                      # Handles raw string input,its the method of preprocessing inside the model
    model,                                # Your trained model
    layers.Activation('sigmoid')            # Optional extra layer
])

export_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer='adam',
    metrics=['accuracy']
)

# Now, you can pass RAW TEXT directly
sample_texts = tf.constant(["This movie was great!",
                            "very good movie",
                            "Terribly good movie"])
predictions = export_model.predict(sample_texts)
print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
[[0.57735217]
 [0.57035446]
 [0.5523901 ]]


#Saving Data

In [71]:
model.save("sentiments.h5")
